# Интро и полезная литература

### Литература

- [Word2vec в картинках](https://habr.com/ru/post/446530/). Статья на хабре от 2019 года.
- [Презентация](http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf) (2017 год) с примерами реализации кода на библиотеке gensim
 
Так как Word2Vec - это уже не просто единичная реализация, а группа алгоритмов для получения векторных представлений слов, то существует масса различных пакетов на эту тему. Мы будем пользоваться gensim.

- [Gensim – Руководство для начинающих](https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/). Здесь нужно изучить ряд терминов, чтобы понимать принцип работы библиотеки. Статья 2019 года.
- [A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model](https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92). Статья 2019 года. По ней хорошо учиться писать код.

### Термины

- **Токен** обычно означает «слово»
- **Документ** обычно может относиться как к «предложению» так и к «абзацу»
- **Корпус** обычно представляет собой «собрание документов» в виде пакета слов или как его еще называют мешка слов 

# Импорт локальных констант и функций

Вначале перейдём на уровень рутового пути репозитория
(в дальнейшем всегда пишем пути от корня репозитория)

In [1]:
cd ..

/Users/polina/WebstormProjects/ml-bug-detector


Заимпортируем всё, что нам пригодится в дальнейшем

In [2]:
from shared.helpers.word2VecPreprocessing import filter_tokens_fo_Word2Vec
from shared.consts import (
    PATH_TO_WORD2VEC_MODEL,
    FOLDER_WITH_JSONS, # папка, в которой лежат json c токенами
    TOKENIZED_JSON_NAME_PREFIX # как начинается имя json, в которых лежат токенизированные скрипты
)

Также внешние библиотеки подключим сразу

In [3]:
from os import walk
import pandas as pd
import json

from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
from tqdm.notebook import tqdm

# Готовим данные перед построением модели

Так как токенизация была разбита на микрозадачи (чтобы хватило мощностей компьютера), то и json у нас много теперь.

In [4]:
# filenames - Список всех файлов в интересуемой нами директории
fdirpath, dirnames, filenames = list(walk(FOLDER_WITH_JSONS))[0]

tokenized_scripts_jsons_paths = list(
    filter(
        lambda script_name: script_name.startswith(TOKENIZED_JSON_NAME_PREFIX),
        filenames
    )
)

Можно было бы пытаться объединить их все в одну, но опять же тут в зависимости от мощностей компьютера, может просто не хватить оперативки держать столько данных в памяти.

Благо word2Vec может учить постепенно: вначале на одних данных, потом "доучить" другим набором данных.

Поэтому вначале учим на первом наборе данных, а потом доучиваем оставшимися постепенно.

**ВАЖНОЕ УТОЧНЕНИЕ:**
Формируется словарь только на первом этапе обучения. Дальше он будет фиксированнным и будут лишь уточняться веса при online learning / resume learning.

In [5]:
def transform_json_file_to_train_data(path_to_json_file):
    tokenized_scripts_dic = {}
    
    tokenizedScriptsJson = open(path_to_json_file)
    raw_tokenized_scripts_dictionary = json.load(tokenizedScriptsJson)

    for fileName in raw_tokenized_scripts_dictionary.keys():
        tokenized_scripts_dic[fileName] = filter_tokens_fo_Word2Vec(raw_tokenized_scripts_dictionary[fileName])
        
    return list(tokenized_scripts_dic.values())

In [6]:
NUMBER_JSON_FILES_TO_FORM_VOCABULARY = 3
tokenized_scripts_list_of_lists = []

for json_index in tqdm(range(NUMBER_JSON_FILES_TO_FORM_VOCABULARY)):
    train_json_path = f"{FOLDER_WITH_JSONS}/{tokenized_scripts_jsons_paths[0]}"

    tokenized_scripts_list_of_lists = [
        *tokenized_scripts_list_of_lists,
        *transform_json_file_to_train_data(train_json_path)
    ]

In [7]:
len(tokenized_scripts_list_of_lists)

1500

# Build Word2Vec model

Теперь бы нужно построить модель и настроить гиперпараметры.

[Эта статья](https://habr.com/ru/post/446530/) говорит, что embedding_size устанавливают обычно на 300.

In [8]:
word2VecModel = Word2Vec(
    # list of lists. Например: [["cat", "say", "meow"], ["dog", "say", "woof"]]
    sentences=tokenized_scripts_list_of_lists,
    
    # The number of dimensions of the embeddings and the default is 100
    size=300, 
    
    #The maximum distance between a target word and words around the target word. The default window is 5.
    window=10,
    
    # финальный размер словаря
    max_final_vocab=50000, 
    
    # игнорировать слова с частотностью ниже, чем эта (регулируется автоматически, если установлено max_final_vocab)
    min_count = 5, 
    
    workers=cpu_count(),
    
    # The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW
    sg=0
)

Смотрим, какие наиболее близкие вектора будут к функции с названием **setTimeout** (это встроенная функция в js, которая на вход принимает функцию первым аргументом, которую нужно исполнить через заданное кол-во времени в миллисекундах, которые будут переданы вторым аргументом).

Число напротив каждого слова - процент близости векторов.

In [9]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('clearTimeout', 0.9035370349884033),
 ('timeout', 0.8836937546730042),
 ('off', 0.8403403759002686),
 ('wait', 0.8383508920669556),
 ('cancel', 0.823870062828064),
 ('errorTimer', 0.8155932426452637),
 ('close', 0.81069415807724),
 ('_self', 0.7997608184814453),
 ('play', 0.7961515784263611),
 ('open', 0.7959170937538147),
 ('render', 0.7932193279266357),
 ('once', 0.7924019694328308),
 ('debounce', 0.7916122674942017),
 ('disableUntilTick', 0.788800835609436),
 ('sender', 0.7858836650848389),
 ('bind', 0.7849777936935425),
 ('destroy', 0.7844083309173584),
 ('dragOverTimeout', 0.7831509113311768),
 ('$timeout', 0.7827353477478027),
 ('schedule', 0.7814319133758545)]

Видим, что модель на маленьком кусочке данных ($\frac{1}{43}$ от всех токенов) показывает крайне странные результаты. Да, в каждом слове можно "натянуть" смысл, но это все будет не то.

Посмотрим лучше, какие будут результаты у модели, когда она "доучиться" на остальных данных.

# Resume learning

API библиотеки gensim предоставил возможность продолжать обучение модели на новых данных

In [10]:
additional_data_jsons = tokenized_scripts_jsons_paths[NUMBER_JSON_FILES_TO_FORM_VOCABULARY:]

for jsonPathIndex in tqdm(range(len(additional_data_jsons))):
    try:
        new_data = transform_json_file_to_train_data(f"{FOLDER_WITH_JSONS}/{additional_data_jsons[jsonPathIndex]}")
    except:
        print('Не получилось расспарсить json:', additional_data_jsons[jsonPathIndex])
        continue
    
    word2VecModel.build_vocab(new_data, update=True)
    word2VecModel.train(new_data, total_examples=word2VecModel.corpus_count, epochs=word2VecModel.epochs)

In [11]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('setInterval', 0.7481271028518677),
 ('clearInterval', 0.7013082504272461),
 ('clearTimeout', 0.7007315158843994),
 ('2000', 0.6274374723434448),
 ('delay', 0.6208372712135315),
 ('timerId', 0.6147360801696777),
 ('500', 0.6064625978469849),
 ('loadMoreIfNeeded', 0.591736912727356),
 ('interval', 0.5881319046020508),
 ('timer', 0.5739376544952393),
 ('5000', 0.5733336806297302),
 ('cancel', 0.5664186477661133),
 ('timeout', 0.5661569833755493),
 ('10000', 0.5637098550796509),
 ('resize', 0.554633378982544),
 ('startTimer', 0.5520493984222412),
 ('focus', 0.5508633852005005),
 ('changeTimer', 0.5498732328414917),
 ('prepareSelectAllHack', 0.5445976257324219),
 ('2500', 0.5437654256820679)]

Результаты стали гораздо более впечатляющими!

- clearTimeout - обрывает запланированный вызов функции
- timeout - через какое время исполнить функцию (один из агрументов всегда)

# Сохранение и загрузка модели

In [12]:
word2VecModel.save(PATH_TO_WORD2VEC_MODEL)

In [13]:
from gensim.models.word2vec import Word2Vec
from shared.consts import PATH_TO_WORD2VEC_MODEL

word2VecModel = Word2Vec.load(PATH_TO_WORD2VEC_MODEL)

In [14]:
word2VecModel.wv.most_similar('setTimeout', topn=20)

[('setInterval', 0.7481271028518677),
 ('clearInterval', 0.7013082504272461),
 ('clearTimeout', 0.7007315158843994),
 ('2000', 0.6274374723434448),
 ('delay', 0.6208372712135315),
 ('timerId', 0.6147360801696777),
 ('500', 0.6064625978469849),
 ('loadMoreIfNeeded', 0.591736912727356),
 ('interval', 0.5881319046020508),
 ('timer', 0.5739376544952393),
 ('5000', 0.5733336806297302),
 ('cancel', 0.5664186477661133),
 ('timeout', 0.5661569833755493),
 ('10000', 0.5637098550796509),
 ('resize', 0.554633378982544),
 ('startTimer', 0.5520493984222412),
 ('focus', 0.5508633852005005),
 ('changeTimer', 0.5498732328414917),
 ('prepareSelectAllHack', 0.5445976257324219),
 ('2500', 0.5437654256820679)]

In [17]:
print('В нашем словаре теперь', len(word2VecModel.wv.vocab), 'слов')

В нашем словаре теперь 122570 слов
